In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.nn import functional as F
from tqdm import tqdm
import re
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word
import nltk
import re
import os
import sys
import seaborn as sns


In [ ]:
data = pd.read_csv('/content/drug.com 2.csv')
data


,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,5/20/2012,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,4/27/2010,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,12/14/2009,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,11/3/2015,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,11/27/2016,37
...,...,...,...,...,...,...,...
4212,84100,Ethinyl estradiol / norgestimate,Birth Control,"""This will be the 6th month I&#039;ve been on ...",5,4/9/2017,6
4213,146853,Alprazolam,Anxiety,"""I suffer from complicated grief and PTSD, I l...",8,1/8/2017,30
4214,124779,Omega-3 polyunsaturated fatty acids,Hypertriglyceridemia,"""I&#039;ve used Lovaza for about 8 months and ...",10,6/22/2008,140
4215,13778,Amphetamine / dextroamphetamine,ADHD,"""I used to take a high dosage time release tab...",9,10/29/2008,19


In [ ]:
!pip install nltk

In [ ]:
!pip install stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#lowercasing first 10 texts
data['Reviews'] = data['review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['Reviews'] = data['review'].str.replace('[^\w\s]',"")
data.review

<ipython-input-12-802c60a31241>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Reviews'] = data['review'].str.replace('[^\w\s]',"")


0       "It has no side effect, I take it in combinati...
1       "My son is halfway through his fourth week of ...
2       "I used to take another oral contraceptive, wh...
3       "This is my first time using any form of birth...
4       "Suboxone has completely turned my life around...
                              ...                        
4212    "This will be the 6th month I&#039;ve been on ...
4213    "I suffer from complicated grief and PTSD, I l...
4214    "I&#039;ve used Lovaza for about 8 months and ...
4215    "I used to take a high dosage time release tab...
4216    "I started phentermine in 2015 weighing in at ...
Name: review, Length: 4217, dtype: object

In [ ]:
#removing stopwords
stop = stopwords.words('english')
data['Reviews'] = data['review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data.review

0       "It has no side effect, I take it in combinati...
1       "My son is halfway through his fourth week of ...
2       "I used to take another oral contraceptive, wh...
3       "This is my first time using any form of birth...
4       "Suboxone has completely turned my life around...
                              ...                        
4212    "This will be the 6th month I&#039;ve been on ...
4213    "I suffer from complicated grief and PTSD, I l...
4214    "I&#039;ve used Lovaza for about 8 months and ...
4215    "I used to take a high dosage time release tab...
4216    "I started phentermine in 2015 weighing in at ...
Name: review, Length: 4217, dtype: object

In [ ]:
#lemmatizing
data['Reviews'] = data['review'].apply(lambda x: " ".join([Word(word).lemmatize()
for word in x.split()]))
data.review

0       "It has no side effect, I take it in combinati...
1       "My son is halfway through his fourth week of ...
2       "I used to take another oral contraceptive, wh...
3       "This is my first time using any form of birth...
4       "Suboxone has completely turned my life around...
                              ...                        
4212    "This will be the 6th month I&#039;ve been on ...
4213    "I suffer from complicated grief and PTSD, I l...
4214    "I&#039;ve used Lovaza for about 8 months and ...
4215    "I used to take a high dosage time release tab...
4216    "I started phentermine in 2015 weighing in at ...
Name: review, Length: 4217, dtype: object

In [ ]:
# Assume binary classification (positive/negative sentiment)
# You may need to adjust the labels based on your dataset
df['label'] = df['rating'].apply(lambda x: 1 if x >= 5 else 0)



In [ ]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)




In [ ]:
# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize the input data
def tokenize_data(data, max_length=128):
    input_ids = []
    attention_masks = []

    for review in data['review']:
        encoded_data = tokenizer.encode_plus(
            review,
            max_length=max_length,
            add_special_tokens=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_data['input_ids'])
        attention_masks.append(encoded_data['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(data['label'].values)

    return input_ids, attention_masks, labels



In [ ]:
# Tokenize the training and testing data
train_input_ids, train_attention_masks, train_labels = tokenize_data(train_df)
test_input_ids, test_attention_masks, test_labels = tokenize_data(test_df)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
batch_size = 32
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss}")



Epoch 1/3: 100%|██████████| 106/106 [1:11:01<00:00, 40.20s/it]


Epoch 1/3, Average Training Loss: 0.5168747691051016


Epoch 2/3: 100%|██████████| 106/106 [1:10:33<00:00, 39.94s/it]


Epoch 2/3, Average Training Loss: 0.333206426422551


Epoch 3/3:  53%|█████▎    | 56/106 [37:59<33:42, 40.46s/it]

In [ ]:
# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted_labels = torch.max(logits, dim=1)
        predictions.extend(predicted_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy:{accuracy}')

Accuracy:0.8509615384615384
